# Advent of Code - D05: If You Give A Seed A Fertilizer

@author: Camillo Moschner

# Import Staments

In [1]:
import re
from tqdm.notebook import tqdm

# Solve Part 1

## Load Data

In [2]:
test_data ="""seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

In [3]:
with open('D05ab_input.txt', 'r') as file:
    puzzle_input_BOTH = file.read()

In [4]:
def map_to_next_category(input_no_list: list, map_line_list: list) -> list:
    """ Using a hashmap -> time complexity = Linear Time - O(n) ==> doesn't scale well, large input ranges will crash normal computers
    """
    current_dict = {}
    for line in [[int(seed_id) for seed_id in re.findall(r'\d+', line)] for line in map_line_list]:
        dest, source, rang = line
        source_range = list(range(source, source+rang,1))
        dest_range = list(range(dest, dest+rang))
        # Generate mapping dictionary
        for idx, x in enumerate(source_range):
            current_dict[x] = dest_range[idx]
    # print(current_dict)
    return [current_dict.get(val, val) for val in input_no_list]

## Binary search approach
Linear Time - O(log n)

In [5]:
def binary_search_for_index(start: int, end: int, target: int) -> int:
    left = 0
    right = end - start  # Total number of elements assuming step size of 1

    while left <= right:
        mid = (left + right) // 2
        mid_value = start + mid  # Calculate the value at this hypothetical index

        if mid_value == target:
            return mid  # Target value's index found
        elif mid_value < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1  # Target not found

In [6]:
binary_search_for_index(0, 11, 53)

-1

In [7]:
def map_to_next_category_wo_memory_allocation(input_no_list: list, map_line_list: list) -> list:
    """
    """
    results = []
    for input_no in input_no_list:
        # print(input_no)
        input_no_results = []
        for line in [[int(seed_id) for seed_id in re.findall(r'\d+', line)] for line in map_line_list]:
            dest_start, source_start, rang = line
            source_end, dest_end = source_start+rang+1, dest_start+rang+1
            # CRUX: identify mapping index across both ranges
            cross_category_idx = binary_search_for_index(source_start, source_end, input_no)
            # print(f"{input_no}, source={source_start}->{source_end}, dest={dest_start}->{dest_end}, idx={cross_category_idx}")
            input_no_results.append(cross_category_idx)
            if cross_category_idx != -1:
                next_cat_val = range(dest_start,dest_end)[cross_category_idx]
                break
        if all([x==-1 for x in input_no_results]):
            results.append(input_no)
        else:
            results.append(next_cat_val)

    return results

In [8]:
map_to_next_category_wo_memory_allocation( [81, 53, 57, 52], ['49 53 8','0 11 42','42 0 7','57 7 4'])

[81, 49, 53, 41]

In [9]:
def process_information_through_maps(input_str: str) -> int:
    # Read data input
    number_conversion_list = [int(seed_id) for seed_id in re.findall(r'\d+', [map_item for map_item in input_str.split('\n\n')][0].split(': ')[-1])]
    almanac = [[map_item.split(':')[0],map_item.split(':\n')[1].splitlines()] for map_item in [map_item for map_item in input_str.split('\n\n')][1:]]
    # Read maps and sequentially convert category to category
    print(f"Starting with seeds {number_conversion_list}\n")
    for idx, current_map in enumerate(almanac[:]):
        print(current_map[0])
        
        number_conversion_list = map_to_next_category_wo_memory_allocation(number_conversion_list, current_map[-1])
        # print(f" {idx} - {current_map[0].split(' ')[0].split('-')[-1]} numbers = {number_conversion_list}\n")
    return f"Lowest location number = {min(number_conversion_list)}"

In [10]:
%%time 
process_information_through_maps(test_data)

Starting with seeds [79, 14, 55, 13]

seed-to-soil map
soil-to-fertilizer map
fertilizer-to-water map
water-to-light map
light-to-temperature map
temperature-to-humidity map
humidity-to-location map
CPU times: user 419 µs, sys: 50 µs, total: 469 µs
Wall time: 448 µs


'Lowest location number = 35'

In [11]:
%%time 
process_information_through_maps(puzzle_input_BOTH)

Starting with seeds [202517468, 131640971, 1553776977, 241828580, 1435322022, 100369067, 2019100043, 153706556, 460203450, 84630899, 3766866638, 114261107, 1809826083, 153144153, 2797169753, 177517156, 2494032210, 235157184, 856311572, 542740109]

seed-to-soil map
soil-to-fertilizer map
fertilizer-to-water map
water-to-light map
light-to-temperature map
temperature-to-humidity map
humidity-to-location map
CPU times: user 5.67 ms, sys: 1.11 ms, total: 6.78 ms
Wall time: 7.13 ms


'Lowest location number = 318728750'

# Solve Part 2

## Load Data

In [7]:
from copy import deepcopy

In [8]:
test_data ="""Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
"""

In [9]:
import numpy as np

In [10]:
def calculate_match_numbers(scratch_card_text_input: str) -> int:
    """
    """
    data_prepocess = [numbers.split('|') for numbers in [card.split(': ')[-1] for card in scratch_card_text_input.splitlines()]]
    match_count = []
    for card in data_prepocess:
        winning_numbers, numbers_owned = set([int(numb) for numb in re.findall(r'\d+', card[0])]), set([int(numb) for numb in re.findall(r'\d+', card[-1])])
        no_matches = len(winning_numbers & numbers_owned)
        match_count.append(no_matches)
    return match_count

In [11]:
def calculate_recursive_winning_number(input_data_str: str) -> int:
    matches_per_game_list = [[x] for x in calculate_match_numbers(input_data_str)]
    copy_tracker = np.ones(len(matches_per_game_list))

    for idx in range(len(matches_per_game_list)):
        matches_no = matches_per_game_list[idx]
        # print(idx+1, matches_per_game_list)
        # original
        for addition in matches_no:
            for counter in range(1, matches_no[0]+1):
                # print(f" idx-{idx} -> addcopyto:{idx+counter+1}")
                matches_per_game_list[idx+counter].append(matches_per_game_list[idx+counter][0])
                copy_tracker[idx+counter] += 1
    return copy_tracker.sum()

In [12]:
calculate_recursive_winning_number(test_data)

In [13]:
%%time
calculate_recursive_winning_number(puzzle_input_BOTH)